In [ ]:
import random
import shutil
from shutil import copyfile
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
base_dir  = '/content/drive/compVisionPackage/Agricultural-crops'
os.chdir(base_dir)

drive_list = tf.io.gfile.listdir(base_dir)

drive_labels = len(drive_list)
print(f"Total labels: = {drive_labels}")

Total Class Labels = 30


In [ ]:
def convert_jpeg_to_csv(img):
  arr = np.asarray(img)
  print(arr.shape)
  lst = []
  for row in arr:
    tmp = []
    for col in row:
        tmp.append(str(col))
    lst.append(tmp)
  with open('my_file.csv', 'w') as f:
    for row in lst:
        f.write(','.join(row) + '\n')

In [ ]:
def svm(W, X, Y):
    reg_strength = 10000  

    N = X.shape[0]
    distances = 1 - Y * (np.dot(X, W))
    distances[distances < 0] = 0 
    hinge_loss = reg_strength * (np.sum(distances) / N)

    cost = 1 / 2 * np.dot(W, W) + hinge_loss
    return cost

In [ ]:


import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split as tts


learning_rate = 0.000001

def init():
    
    for file in os.listdir(base_dir):
      if file.endswith(".jpeg"):
        convert_jpeg_to_csv(file)


    data = pd.read_csv('./data.csv')
    collect_map = {'M': 1, 'B': -1}
    data['diagnosis'] = data['diagnosis'].map(collect_map)
    data.drop(data.columns[[-1, 0]], axis=1, inplace=True)

    Y = data.loc[:, 'diagnosis']
    X = data.iloc[:, 1:]


    X_train, X_test, y_train, y_test = tts(X, Y, test_size=0.1, train_size=0.9, random_state=42)

    weights = np.zeros(X_train.shape[1])
    cost = svm(weights, features, outputs)
    print("Cost is: {}".format(cost))
    print("training finished.")
    print("weights are: {}".format(W))


def sgd(features, outputs):
    max_epochs = 5000
    weights = np.zeros(features.shape[1])
    nth = 0
    prev_cost = float("inf")
    cost_threshold = 0.01 
    for epoch in range(1, max_epochs):
        X, Y = shuffle(features, outputs)
        for ind, x in enumerate(X):
            ascent = calculate_cost_gradient(weights, x, Y[ind])
            weights = weights - (learning_rate * ascent)
        if epoch == 2 ** nth or epoch == max_epochs - 1:
            cost = compute_cost(weights, features, outputs)
            print("Epoch is:{} and Cost is: {}".format(epoch, cost))
            if abs(prev_cost - cost) < cost_threshold * prev_cost:
                return weights
            prev_cost = cost
            nth += 1
    return weights

In [ ]:
def softmax(z):
    z -= np.max(z)
    return np.exp(z) / np.sum(np.exp(z))

In [ ]:
def ridge_regression(X, y, alpha=0.01, lambda_value=1, epochs=30):
 
    m = np.shape(X)[0]
    n = np.shape(X)[1]
 
    X = np.concatenate((np.ones((m, 1)), X), axis=1)
    W = np.random.randn(n + 1, )
 
    cost_history_list = []
 
    for current_iteration in range(epochs):
 
        y_estimated = X.dot(W)
 
        error = y_estimated - y
 
        ridge_reg_term = (lambda_value / 2 * m) * np.sum(np.square(W))
 
        cost = (1 / 2 * m) * np.sum(error ** 2) + ridge_reg_term
 
        gradient = (1 / m) * (X.T.dot(error) + (lambda_value * W))
 
        W = W - alpha * gradient
 
        print(f"cost:{cost} \t iteration: {current_iteration}")
 
        cost_history_list.append(cost)
 
    return W, cost_history_list